In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly_express as px
import seaborn as sns

palette = sns.color_palette("rainbow", 8)


import warnings
warnings.filterwarnings("ignore")


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #importing basic visualization libraries
import seaborn as sns  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/sample_submission.csv
/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/labels.csv.zip
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/labels.csv
/kaggle/input/description.md
/kaggle/input/GCP-Coupons-Instructions.rtf
/kaggle/input/sample_submission.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/labels.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv
/kaggle/input/new-york-city-taxi-fare-prediction/labels.csv
/kaggle/input/new-york-city-taxi-fare-prediction/description.md
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv.zip


In [4]:
#read data
df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', nrows=2_000_000)
test_data = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 122.1+ MB


In [6]:
#df.isnull().sum()


In [7]:
# 删除具有空值的行
df.dropna(subset=['dropoff_longitude', 'dropoff_latitude'], inplace=True)


In [8]:
#df.isnull().sum()


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1999986 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 137.3+ MB


In [10]:
df = df[(df['fare_amount'] >= 2.5) & (df['passenger_count'] > 0) & (df['passenger_count']  5)]


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2402454032.py, line 1)

In [11]:
# 检查 fare_amount 和 passenger_count 列的值是否都大于0
fare_amount_greater_than_zero = (df['fare_amount'] > 0).all()
passenger_count_greater_than_zero = (df['passenger_count'] > 0).all()

if fare_amount_greater_than_zero and passenger_count_greater_than_zero:
    print("All values in fare_amount and passenger_count are greater than 0.")
else:
    print("There are values in fare_amount or passenger_count that are not greater than 0.")


There are values in fare_amount or passenger_count that are not greater than 0.


In [12]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1999986 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 137.3+ MB


In [13]:
df.describe()


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,1.999986e+06,1.999986e+06,1.999986e+06,1.999986e+06,1.999986e+06,1.999986e+06
mean,1.134768e+01,-7.252324e+01,3.992965e+01,-7.252395e+01,3.992808e+01,1.684125e+00
std,9.852609e+00,1.286798e+01,7.983330e+00,1.277497e+01,1.032382e+01,1.314979e+00
min,-6.200000e+01,-3.377681e+03,-3.458665e+03,-3.383297e+03,-3.461541e+03,0.000000e+00
25%,6.000000e+00,-7.399208e+01,4.073491e+01,-7.399141e+01,4.073400e+01,1.000000e+00
50%,8.500000e+00,-7.398181e+01,4.075263e+01,-7.398016e+01,4.075312e+01,1.000000e+00
75%,1.250000e+01,-7.396713e+01,4.076710e+01,-7.396369e+01,4.076809e+01,2.000000e+00
max,1.273310e+03,2.856442e+03,2.621628e+03,3.414307e+03,3.345917e+03,2.080000e+02


In [14]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format= "%Y-%m-%d %H:%M:%S UTC")

# 2009-06-15 17:26:21 UTC
# add time information
df['year'] = df.pickup_datetime.apply(lambda t: t.year)
df['month'] = df.pickup_datetime.apply(lambda t: t.month)
df['weekday'] = df.pickup_datetime.apply(lambda t: t.weekday())
df['hour'] = df.pickup_datetime.apply(lambda t: t.hour)


In [15]:
df


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9


In [16]:
def distance(lat1, lon1, lat2, lon2):
    
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...


In [17]:
df['distance'] = distance(df.pickup_latitude, df.pickup_longitude, 
                                    df.dropoff_latitude, df.dropoff_longitude)


In [18]:
df


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [19]:
Df=df
Df


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [20]:
Df=Df.drop(['pickup_datetime','key'], axis=1)
Df


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,0,17,0.640487
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
...,...,...,...,...,...,...,...,...,...,...,...
1999995,4.0,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,7.0,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,10.5,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,10.9,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [21]:
# Assuming Df is your DataFrame

# Define boundaries based on the provided xlim and ylim
xlim = [-74.03, -73.85]
ylim = [40.70, 40.85]

# Filter data within the specified coordinates
Df = Df[
    (Df['pickup_latitude'].between(ylim[0], ylim[1])) &
    (Df['pickup_longitude'].between(xlim[0], xlim[1])) &
    (Df['dropoff_latitude'].between(ylim[0], ylim[1])) &
    (Df['dropoff_longitude'].between(xlim[0], xlim[1]))
]


In [22]:
zero_distance_rows = Df[Df['distance'] == 0]
zero_distance_rows.count()


fare_amount          17628
pickup_longitude     17628
pickup_latitude      17628
dropoff_longitude    17628
dropoff_latitude     17628
passenger_count      17628
year                 17628
month                17628
weekday              17628
hour                 17628
distance             17628
dtype: int64

In [23]:
# 查看經緯度資料的範圍
print("\nPickup Latitude:")
print(Df['pickup_latitude'].describe())

print("\nPickup Longitude:")
print(Df['pickup_longitude'].describe())

print("\nDropoff Latitude:")
print(Df['dropoff_latitude'].describe())

print("\nDropoff Longitude:")
print(Df['dropoff_longitude'].describe())



Pickup Latitude:
count    1.813277e+06
mean     4.075377e+01
std      2.164448e-02
min      4.070000e+01
25%      4.073900e+01
50%      4.075484e+01
75%      4.076827e+01
max      4.085000e+01
Name: pickup_latitude, dtype: float64

Pickup Longitude:
count    1.813277e+06
mean    -7.397883e+01
std      2.310766e-02
min     -7.402995e+01
25%     -7.399232e+01
50%     -7.398220e+01
75%     -7.396939e+01
max     -7.385009e+01
Name: pickup_longitude, dtype: float64

Dropoff Latitude:
count    1.813277e+06
mean     4.075466e+01
std      2.351332e-02
min      4.070000e+01
25%      4.073922e+01
50%      4.075537e+01
75%      4.076908e+01
max      4.084999e+01
Name: dropoff_latitude, dtype: float64

Dropoff Longitude:
count    1.813277e+06
mean    -7.397733e+01
std      2.402810e-02
min     -7.402999e+01
25%     -7.399171e+01
50%     -7.398107e+01
75%     -7.396723e+01
max     -7.385000e+01
Name: dropoff_longitude, dtype: float64


In [24]:
Df


,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour,distance
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,1,16,5.250670
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,3,0,0.863411
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,5,4,1.739386
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,1,7,1.242218
5,12.1,-74.000964,40.731630,-73.972892,40.758233,1,2011,1,3,9,2.353281
...,...,...,...,...,...,...,...,...,...,...,...
1999995,4.0,-73.950787,40.770836,-73.951043,40.774543,1,2014,11,0,13,0.256479
1999996,7.0,-73.983035,40.748276,-73.974725,40.741978,1,2013,2,0,17,0.615289
1999997,10.5,-74.003482,40.743577,-73.976758,40.765265,1,2013,7,4,22,2.049851
1999998,10.9,-73.971819,40.797243,-73.985921,40.752459,3,2009,1,5,9,3.181034


In [25]:
# now removing the key and pickup_datetime coloumn from the dataset
Df = Df.drop(['pickup_longitude', 'pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)


In [26]:
Df


,fare_amount,passenger_count,year,month,weekday,hour,distance
1,16.9,1,2010,1,1,16,5.250670
2,5.7,2,2011,8,3,0,0.863411
3,7.7,1,2012,4,5,4,1.739386
4,5.3,1,2010,3,1,7,1.242218
5,12.1,1,2011,1,3,9,2.353281
...,...,...,...,...,...,...,...
1999995,4.0,1,2014,11,0,13,0.256479
1999996,7.0,1,2013,2,0,17,0.615289
1999997,10.5,1,2013,7,4,22,2.049851
1999998,10.9,3,2009,1,5,9,3.181034


In [27]:
# 將特定列轉換為類別型態
columns_to_convert = ['year', 'month', 'weekday', 'hour']

for col in columns_to_convert:
    Df[col] = Df[col].astype('category')

# 查看轉換後的 DataFrame
print(Df.dtypes)


fare_amount         float64
passenger_count       int64
year               category
month              category
weekday            category
hour               category
distance            float64
dtype: object


In [28]:
Df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1813277 entries, 1 to 1999999
Data columns (total 7 columns):
 #   Column           Dtype   
---  ------           -----   
 0   fare_amount      float64 
 1   passenger_count  int64   
 2   year             category
 3   month            category
 4   weekday          category
 5   hour             category
 6   distance         float64 
dtypes: category(4), float64(2), int64(1)
memory usage: 62.3 MB


In [29]:
Df.isnull().sum()


fare_amount        0
passenger_count    0
year               0
month              0
weekday            0
hour               0
distance           0
dtype: int64

In [30]:
#Df=Df.sample(100000)


In [31]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


In [32]:
# 假设你的 DataFrame 名称为 'df'
# 绘制数据集中所有数值型特征的箱线图
#plt.figure(figsize=(10, 6))  # 可选：设置图形大小
#sns.boxplot(data=Df, orient='h')  # orient='h'表示水平放置箱线图
#plt.title('Boxplot of Numerical Features')  # 设置图标题
#plt.xlabel('Values')  # 设置 x 轴标签
#plt.show()


In [33]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.boxplot(x='month', y='fare_amount', data=Df)
#plt.title('Boxplot of fare_amount Data by month')
#plt.xlabel('month')
#plt.ylabel('fare_amount')
#plt.show()


In [34]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.barplot(x='month', y='fare_amount', data=Df)
#plt.title('Barplot of fare_amount Data by month')
#plt.xlabel('month')
#plt.ylabel('fare_amount')
#plt.show()


In [35]:
import seaborn as sns
import matplotlib.pyplot as plt


#plt.figure(figsize=(8, 6))
#sns.barplot(x='weekday', y='fare_amount', data=Df)
#plt.title('Barplot of fare_amount by weekday')
#plt.xlabel('weekday')
#plt.ylabel('fare_amount')
#plt.show()


In [36]:
import seaborn as sns
import matplotlib.pyplot as plt

# 使用 regplot 绘制 fare_amount 和 distance 之间的关系图
#plt.figure(figsize=(8, 6))  # 设置图形大小（可选）
#sns.regplot(x='passenger_count', y='fare_amount', data=Df, scatter_kws={'alpha':0.3})  # 绘制 regplot
#plt.title('Relationship between fare_amount and distance')  # 设置图标题
#plt.xlabel('Distance')  # 设置 x 轴标签
#plt.ylabel('Fare Amount')  # 设置 y 轴标签
#plt.show()


In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

# 创建散点图
#sns.scatterplot(x='distance', y='fare_amount', data=Df)

# 设置图表标题和轴标签
#plt.title('Relationship between Fare Amount and Distance')
#plt.xlabel('Distance')
#plt.ylabel('Fare Amount')
#plt.figure(figsize=(8, 6))  # 设置图形大小（可选）

# 显示图表
#plt.show()


In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# 假設你的 DataFrame 名稱為 df
#columns_to_visualize = ['fare_amount', 'passenger_count', 'year', 'month', 'weekday', 'hour', 'distance']
#mpl.rcParams.update({'font.size': 16})  # 調整字體大小

# 使用 Seaborn 的 pairplot 進行視覺化

#sns.pairplot(Df[columns_to_visualize])

#plt.show()


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 假設資料存在名為 df 的 DataFrame 中

# 提取特徵和目標值
X = Df.drop(columns=['fare_amount'])
y = Df['fare_amount']

# 分割資料為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立並訓練線性回歸模型
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# 預測測試集
linear_predictions = linear_model.predict(X_test)

# 計算線性回歸模型的均方誤差
linear_mse = mean_squared_error(y_test, linear_predictions)
print(f"線性回歸模型的均方誤差 (MSE): {linear_mse}")
# 假設 linear_mse 是先前計算得到的均方誤差
linear_rmse = mean_squared_error(y_test, linear_predictions, squared=False)
print(f"線性回歸模型的均方根誤差 (RMSE): {linear_rmse}")


線性回歸模型的均方誤差 (MSE): 13.939940705665556
線性回歸模型的均方根誤差 (RMSE): 3.7336229999379364


In [40]:
from sklearn.metrics import accuracy_score

# 轉換預測值為二元分類
threshold = 10  # 可以根據情況調整閾值
binary_predictions = (linear_predictions >= threshold).astype(int)

# 計算準確率
accuracy = accuracy_score((y_test >= threshold).astype(int), binary_predictions)
print(f"分類準確率 (Accuracy): {accuracy}")


分類準確率 (Accuracy): 0.868544846907262


In [41]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# 假設資料存在名為 df 的 DataFrame 中

# 要編碼的欄位名稱
categorical_columns = ['year', 'month', 'weekday', 'hour']

# 使用 LabelEncoder 將類別型資料轉換為數值型
label_encoder = LabelEncoder()
for col in categorical_columns:
    Df[col] = label_encoder.fit_transform(Df[col])

# 接下來建立 XGBoost 模型和訓練，使用前面提到的程式碼

# 假設資料存在名為 df 的 DataFrame 中

# 提取特徵和目標值
X = Df.drop(columns=['fare_amount'])
y = Df['fare_amount']

# 分割資料為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 建立XGBoost回歸模型
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

# 訓練XGBoost模型
xgb_model.fit(X_train, y_train)

# 預測測試集
xgb_predictions = xgb_model.predict(X_test)

# 計算XGBoost模型的均方誤差
xgb_mse = mean_squared_error(y_test, xgb_predictions)
print(f"XGBoost模型的均方誤差 (MSE): {xgb_mse}")
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
print(f"XGB模型的均方根誤差 (RMSE): {xgb_rmse}")


XGBoost模型的均方誤差 (MSE): 14.890087356242203
XGB模型的均方根誤差 (RMSE): 3.858767595520907


In [42]:
# 設定閾值來轉換成二元分類
threshold = 10  # 可以根據實際情況調整閾值
binary_predictions = (xgb_predictions >= threshold).astype(int)

# 轉換測試集的目標值為二元分類
binary_actual = (y_test >= threshold).astype(int)

# 計算準確率
accuracy = accuracy_score(binary_actual, binary_predictions)
print(f"準確率 (Accuracy): {accuracy}")


準確率 (Accuracy): 0.8717076237536399


In [43]:
# 將 pickup_datetime 轉換為 datetime 格式
test_data['pickup_datetime'] = pd.to_datetime(test_data['pickup_datetime'])

# 提取年、月、星期幾、幾點資訊
test_data['year'] = test_data['pickup_datetime'].dt.year.astype('category')
test_data['month'] = test_data['pickup_datetime'].dt.month.astype('category')
test_data['weekday'] = test_data['pickup_datetime'].dt.dayofweek.astype('category')
test_data['hour'] = test_data['pickup_datetime'].dt.hour.astype('category')
test_data = test_data.drop(['pickup_datetime'], axis=1)


In [44]:
test_data


,key,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,weekday,hour
0,2010-10-01 21:26:11.0000001,-73.983130,40.761970,-73.994386,40.749236,1,2010,10,4,21
1,2013-10-06 01:38:00.00000083,-73.948505,40.753977,-73.808195,40.731952,2,2013,10,6,1
2,2012-03-30 19:13:53.0000001,-73.973964,40.791979,-73.979018,40.785544,1,2012,3,4,19
3,2012-02-08 02:57:23.0000001,-73.991478,40.738907,-73.907198,40.861572,2,2012,2,2,2
4,2013-12-13 22:56:00.000000237,-73.986281,40.740067,-73.933927,40.856781,2,2013,12,4,22
...,...,...,...,...,...,...,...,...,...,...
9909,2010-12-02 01:44:39.0000002,-73.992667,40.734183,-73.995432,40.739215,4,2010,12,3,1
9910,2010-06-07 16:17:38.0000003,0.000000,0.000000,0.000000,0.000000,1,2010,6,0,16
9911,2013-11-01 08:37:36.0000003,-73.990315,40.746236,-73.991841,40.733516,1,2013,11,4,8
9912,2011-05-03 10:14:00.000000123,-73.956110,40.787588,-73.971877,40.765808,1,2011,5,1,10


In [45]:
Df


,fare_amount,passenger_count,year,month,weekday,hour,distance
1,16.9,1,1,0,1,16,5.250670
2,5.7,2,2,7,3,0,0.863411
3,7.7,1,3,3,5,4,1.739386
4,5.3,1,1,2,1,7,1.242218
5,12.1,1,2,0,3,9,2.353281
...,...,...,...,...,...,...,...
1999995,4.0,1,5,10,0,13,0.256479
1999996,7.0,1,4,1,0,17,0.615289
1999997,10.5,1,4,6,4,22,2.049851
1999998,10.9,3,0,0,5,9,3.181034


In [46]:
test_data['distance'] = distance(test_data.pickup_latitude, test_data.pickup_longitude, 
                                    test_data.dropoff_latitude, test_data.dropoff_longitude)
test_data = test_data.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'], axis=1)
test_data


,key,passenger_count,year,month,weekday,hour,distance
0,2010-10-01 21:26:11.0000001,1,2010,10,4,21,1.058853
1,2013-10-06 01:38:00.00000083,2,2013,10,6,1,7.500968
2,2012-03-30 19:13:53.0000001,1,2012,3,4,19,0.517284
3,2012-02-08 02:57:23.0000001,2,2012,2,2,2,9.553148
4,2013-12-13 22:56:00.000000237,2,2013,12,4,22,8.516409
...,...,...,...,...,...,...,...
9909,2010-12-02 01:44:39.0000002,4,2010,12,3,1,0.376609
9910,2010-06-07 16:17:38.0000003,1,2010,6,0,16,0.000000
9911,2013-11-01 08:37:36.0000003,1,2013,11,4,8,0.882490
9912,2011-05-03 10:14:00.000000123,1,2011,5,1,10,1.716139


In [47]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [48]:
# 假設你有一個名為 df 的 DataFrame，其中包含了訓練資料集

# 假設 df 包含目標值在名為 'target' 的欄位中，並且其他欄位是特徵
# 如果需要，可以分割資料為訓練集和測試集
# train_df, test_df = train_test_split(df, test_size=0.2)

# 從訓練資料集中提取特徵和目標
# 這裡假設 'target' 是目標列
X_train = Df.drop(columns=['fare_amount'])
y_train = Df['fare_amount']

# 如果有必要，對特徵進行標準化或其他前處理
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# 建立並訓練線性回歸模型
model = LinearRegression()
model.fit(X_train, y_train)



# 提取測試集特徵資料
test_features = test_data[['passenger_count','year','month','weekday','hour','distance']]

# 使用模型進行預測
predictions = model.predict(test_features)

# 將預測結果加入測試集資料中（如果需要）
test_data['fare_amount'] = predictions

# 儲存包含預測結果的測試集資料（如果需要）
test_data.to_csv('predicted_test_data_linear_1116.csv', index=False)  # 儲存到一個新的 CSV 檔案中


In [49]:
from sklearn.model_selection import cross_val_score

# 計算 R² score
r2 = model.score(X_train, y_train)
print(f"R² score: {r2}")

# 使用 K-fold 交叉驗證計算 R² score
num_folds = 5  # 定義交叉驗證的 fold 數量
cv_r2_scores = cross_val_score(model, X_train, y_train, cv=num_folds, scoring='r2')

print(f"Cross-validated R² scores: {cv_r2_scores}")
print(f"Mean R² score: {np.mean(cv_r2_scores)}")


R² score: 0.6594935215255799
Cross-validated R² scores: [0.64308531 0.67198107 0.65778807 0.6591515  0.66627616]
Mean R² score: 0.6596564217377272


In [50]:
from sklearn.preprocessing import LabelEncoder

# 從訓練資料集中提取特徵和目標
# 這裡假設 'target' 是目標列
X_train = Df.drop(columns=['fare_amount','year','month','weekday','hour'])
y_train = Df['fare_amount']
# 建立XGBoost回歸模型
model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)

# 訓練模型
model.fit(X_train, y_train)

# 使用模型進行預測
predictions = model.predict(test_features[['passenger_count','distance']])

# 將預測結果加入測試集資料中
test_data['fare_amount'] = predictions
submission=test_data[['key','fare_amount']]
# 儲存包含預測結果的測試集資料
submission.to_csv('submission.csv', index=False)


In [51]:
from sklearn.model_selection import cross_val_score

# 計算 R² score
r2 = model.score(X_train, y_train)
print(f"R² score: {r2}")

# 使用 K-fold 交叉驗證計算 R² score
num_folds = 5  # 定義交叉驗證的 fold 數量
cv_r2_scores = cross_val_score(model, X_train, y_train, cv=num_folds, scoring='r2')

print(f"Cross-validated R² scores: {cv_r2_scores}")
print(f"Mean R² score: {np.mean(cv_r2_scores)}")


R² score: 0.6674326679295509
Cross-validated R² scores: [0.65111226 0.67924972 0.66505241 0.66873645 0.67346341]
Mean R² score: 0.6675228506501887


In [52]:
submission


,key,fare_amount
0,2010-10-01 21:26:11.0000001,7.422479
1,2013-10-06 01:38:00.00000083,30.723536
2,2012-03-30 19:13:53.0000001,5.244805
3,2012-02-08 02:57:23.0000001,34.599270
4,2013-12-13 22:56:00.000000237,34.599270
...,...,...
9909,2010-12-02 01:44:39.0000002,4.985916
9910,2010-06-07 16:17:38.0000003,10.349336
9911,2013-11-01 08:37:36.0000003,6.770314
9912,2011-05-03 10:14:00.000000123,9.820594
